# Test the `cs_tag.Alignment.extract_cs` objects
This notebook is designed to be run with `nbval` as a test.

First, create a `DummyAlignedSegment` class that emulates the necessary part of a `pysam.AlignedSegment`.
It returns all the required attributes for internal functioning of `cs_tag.Alignment`, even if many of these just have dummy values:

In [1]:
class DummyAlignedSegment:
    """Emulates `pysam.AlignedSegment` for testing `cs_tag.Alignment`."""
    
    def __init__(self, *, reference_start, cs):
        # attributes with hardcoded values are irrelevant to test
        self.query_name = 'queryname'
        self.reference_name = 'refname'
        self.query_alignment_start = 0
        self.query_length = 0
        self.query_alignment_end = 0
        self.is_reverse = '+'
        self.reference_end = 0
        self.is_unmapped = False
        
        # relevant attributes
        self.reference_start = reference_start
        self._cs = cs
        
    def get_tag(self, tag):
        if tag == 'cs':
            return self._cs
        else:
            raise ValueError('invalid tag')

We will test `cs_tag.Alignment` objects:

In [2]:
from alignparse.cs_tag import Alignment

First test segment that is all identities:

In [3]:
a_ident = Alignment(DummyAlignedSegment(
                reference_start=0,
                cs=':20'))

In [4]:
a_ident.extract_cs(0, 20)

(':20', 0, 0)

In [5]:
a_ident.extract_cs(5, 26)

(':15', 0, 6)

Another segment that is all identities but starts shifted:

In [6]:
a_ident_shifted = Alignment(DummyAlignedSegment(
                reference_start=5,
                cs=':20'))

In [7]:
a_ident_shifted.extract_cs(0, 20)

(':15', 5, 0)

In [8]:
a_ident_shifted.extract_cs(4, 26)

(':20', 1, 1)

In [9]:
a_ident_shifted.extract_cs(3, 18)

(':13', 2, 0)

In [10]:
a_del = Alignment(DummyAlignedSegment(
                reference_start=0,
                cs='-aatgccgcttcaatgcc'))

In [11]:
a_del.extract_cs(6, 35)

('-gcttcaatgcc', 0, 18)

In [12]:
a_ins_split = Alignment(DummyAlignedSegment(
                reference_start=0,
                cs=':5+atg:3'))

In [13]:
a_ins_split.extract_cs(0, 5)

(':5+atg', 0, 0)